In [8]:
#Multivariate Analysis

#Import libraries
library(ggpubr)
library(dplyr)
library(lme4)
library(ggplot2)
library(rms)
library(mgcv)
library(tidymv)
library("ggthemes")
library(broom)
library(erer)
library(stringr)
library(tidyverse)
library(stringr)
library(R.utils)
library(data.table)

In [9]:
#Restricted analysis (only)

#Read the complete set (no imputation)
UKBB_AG2_m <- fread("~/jupyter/UKBB_AG2_12Jan21.txt", header = TRUE, na.strings=c("",".","NA")) %>% select(f.eid,T2D_status,ALBUMINERIA.0.0,
                                 ESKD.0.0,CKD.0.0,DN.0.0,ALL.0.0,NONESKD.0.0,DNCKD.0.0,
                                 CTRL_DNCKD.0.0,ACR.0.0,EGFR.0.0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,SEX.0.0,
                                 IDEAL_DIET2.0.0,LIFESCORE,AGE.0.0, SES_TDI.0.0,BMI.0.0,EDUYEARS,SBP.0.0,HYP_POS1,STATIN,
                                 WHR.0.0,GRS_WT_LIRd2,GRS_WT_LIRt,GRS_WT_LIRq,GRS_WT_LIRf3,GRS_WT_IRd2,
                                 GRS_WT_IRt,GRS_WT_IRq,GRS_WT_IRf3,GRS_WT_IR53d2,GRS_WT_IR53t,
                                 GRS_WT_IR53q,GRS_WT_IR53f3,GRS_WT_T2DIRd2,GRS_WT_T2DIRt,
                                 GRS_WT_T2DIRq,GRS_WT_T2DIRf3,GRS_WT_L5E8IRd2,GRS_WT_L5E8IRt,
                                 GRS_WT_L5E8IRq,GRS_WT_L5E8IRf3,GRS_WT_L1E5IRd2,GRS_WT_L1E5IRt,
                                 GRS_WT_L1E5IRq,GRS_WT_L1E5IRf3)
UKBB_AG2=as.data.frame(UKBB_AG2_m)
rm(UKBB_AG2_m)

#Import the glucose and hba1c
UKBB_AG_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/UKBB_AG.txt", header = TRUE, na.strings=c("",".","NA")) %>% 
             select(f.eid,GLUCOSE.0.0,HBA1C.0.0)
UKBB_AG=as.data.frame(UKBB_AG_m)
rm(UKBB_AG_m)

UKBB_AG2 <- merge(UKBB_AG,UKBB_AG2,by='f.eid')

In [10]:
RESULTS_CONT <- data.frame()
RESULTS_OR <- data.frame()

##MODEL APPROACH #1 - Three Risk Groups

#Previously evaluated non-linearity in prior section#
#Current section evalautes the continuous form:
RESULTS_CONT <- data.frame()
RESULTS_OR <- data.frame()

for (ii in c('relevel(as.factor(GRS_WT_IRf3),"low risk")',
             'relevel(as.factor(GRS_WT_T2DIRf3),"low risk")',
             'relevel(as.factor(GRS_WT_LIRf3),"low risk")',
             'relevel(as.factor(GRS_WT_L1E5IRf3),"low risk")',
             'relevel(as.factor(GRS_WT_L5E8IRf3),"low risk")')) {
    for(kk in c('Model1')){
          for (jj in c('GLUCOSE.0.0','HBA1C.0.0')) {
              
            #Used across formulas
        term <- ii   
              
        #Model 1 - Age, gender, PCI
        if(kk == "Model1"){
            
        fmla <- as.formula(paste0(jj," ~ ",term, "+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10"), env = environment()) }
        if(kk == "Model2"){
            
        fmla <- as.formula(paste0(jj," ~ ", term,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0"), env = environment()) }
          
        if(kk == "Model3"){
            
        fmla <- as.formula(paste0(jj," ~ ", term,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0 + BMI.0.0"), env = environment()) }
          
        M1 <- glm(fmla, data=UKBB_AG2, gaussian)                                                                                
        M1_2 <-  M1 %>% summary()
        M1_3 <- anova(M1,test="LRT")
        
        #Model 1 - Sub-groups (same model)
        #Type 2 Diabetics
        M1_T2D <- UKBB_AG2 %>% glm(formula=fmla,family=gaussian)                                                                                
        M1_T2D2 <- M1_T2D  %>% summary()
        M1_T2D3 <- anova(M1_T2D ,test="LRT")
            
        #Non-Diabetes
        M1_ND <- UKBB_AG2 %>% glm(formula=fmla, family=gaussian)                                                                                
        M1_ND2 <- M1_ND  %>% summary()
        M1_ND3 <- anova(M1_ND ,test="LRT")
        
        TEMP<- list(model=kk, var=jj,var2=ii, total=M1_2,t2d=M1_T2D2,nd=M1_ND2,lrt_tot=M1_3,
                    lrt_t2d=M1_T2D3,lrt_nd=M1_ND3)
                     
        #Confidence Intervals
        #Entire Sample
        M1$dwn_conf <- coefficients(TEMP$total)[,1]-1.96*coefficients(TEMP$total)[,2]
        M1$up_conf <- coefficients(TEMP$total)[,1]+1.96*coefficients(TEMP$total)[,2]
        TABLE2 <- round(cbind(beta=coefficients(TEMP$total)[,1],CI=cbind(M1$dwn_conf,M1$up_conf)),3)   
        #Diabetics
        M1_T2D$dwn_conf <- coefficients(TEMP$t2d)[,1]-1.96*coefficients(TEMP$t2d)[,2]
        M1_T2D$up_conf <- coefficients(TEMP$t2d)[,1]+1.96*coefficients(TEMP$t2d)[,2]
        TABLE2_T2D <- round(cbind(beta=coefficients(TEMP$t2d)[,1],CI=cbind(M1_T2D$dwn_conf,M1_T2D$up_conf)),3)
        #Non-Diabetics
        M1_ND2$dwn_conf <- coefficients(TEMP$nd)[,1]-1.96*coefficients(TEMP$nd)[,2]
        M1_ND2$up_conf <- coefficients(TEMP$nd)[,1]+1.96*coefficients(TEMP$nd)[,2]
        TABLE2_ND <- round(cbind(beta=coefficients(TEMP$nd)[,1],CI=cbind(M1_ND2$dwn_conf,M1_ND2$up_conf)),3)
            
        #Summary Statistics
        OR_CI <- list(model=kk, var=jj,var2=ii,nd_ci=TABLE2_ND,t2d_ci=TABLE2_T2D,all_ci=TABLE2)
        RESULTS_OR <- c(RESULTS_OR,OR_CI)    
        #Combined results
        RESULTS_CONT <- c(RESULTS_CONT,TEMP)
       }
    }
}

In [11]:
#Total Sample Score

#Intermediate and High Risk (Three Categories)
#Variable of interest listed first
#Only Save Two Coefficients

#6 = unique number of elements in list of 180 objects
#4,5,6 = model-based results (glm summary)
coef_all3 <- data.frame()
for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

  #identifiers
  model_id <- 2 + 9*(i-1) #model id
  i1 <- 4 + 9*(i-1) #total models
  i2 <- 5 + 9*(i-1) #t2d models
  i3 <- 6 + 9*(i-1) #nd models
    
  #lrt
  i7 <- 7 + 9*(i-1) #total models
  i8 <- 8 + 9*(i-1) #t2d models
  i9 <- 9 + 9*(i-1) #nd models 

    
  #model 1,2,3
  term <- i - (i%/%30)*30 + 1 #remainder from subtracting from multiple of 30
  i4 <- ifelse(term %in% c(1:10),
               'Model 1',
               ifelse(term %in% c(11:20),'Model 2',
                      ifelse(term %in% c(21:30),'Model 3')))
  
  #ENTIRE SAMPLE 
  #pull coefficients
  coef_all <- data.frame(round(coefficients(RESULTS_CONT[[i1]]),2)[c(2:10),1])
  id_model <- names(round(coefficients(RESULTS_CONT[[i1]]),2)[c(2:10),1])
  model_id2 <- RESULTS_CONT[[model_id]]
  rep_model <- rep(model_id2,dim(coef_all)[1])
  rep_adj <- rep(i4,dim(coef_all)[1])
  
  #pull coefficients and calculate 95% CI
  up_coef <- data.frame(round(coefficients(RESULTS_CONT[[i1]])[,1]+1.96*coefficients(RESULTS_CONT[[i1]])[,2],2)[2:10])
  down_coef <- data.frame(round(coefficients(RESULTS_CONT[[i1]])[,1]-1.96*coefficients(RESULTS_CONT[[i1]])[,2],2)[2:10])
  #likelihood ratio
  lrt_total <- RESULTS_CONT[[i7]][2,5]
  lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
  coef_all2 <- cbind(rep_model[1],id_model) #model outcome to model var
  coef_all2 <- cbind(coef_all2,coef_all) #coefficients
  coef_all2 <- cbind(coef_all2,rep_adj)
  coef_all2 <- cbind(coef_all2,down_coef) #95 CI
  coef_all2 <- cbind(coef_all2,up_coef) #95 CI
  coef_all2 <- cbind(coef_all2,lrt_total2)
  coef_all3 <- rbind(coef_all3,coef_all2)
}

#reformat table
#rename
names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
#combine HR and 95% CI
coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
coef_all3 <- subset(coef_all3, select = -c(3,5,6))
#high/low
coef_all3$category <- ifelse(str_detect(coef_all3$id_model,"high")==TRUE,"2_high","1_medium")
#substring
coef_all3$sub <- substr(coef_all3$id_model,19,32)
coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
#key
coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
#spread
coef_all3 <- subset(coef_all3)

#Export the Sheet
write.csv2(coef_all3,'/cellar/users/agarduno/jupyter/Analysis/All_GlucHba1c_20Oct21fullrev.txt')